In [5]:
import re
import time
import requests
import requests_cache
from bs4 import BeautifulSoup

requests_cache.install_cache()

In [6]:
url = 'http://vetaugiare24h.com/ve-xe.html'
tree = BeautifulSoup(requests.get(url).text)

In [7]:
to_places = tree.find_all('li', {'id': 'chinhlitrong'})
len(to_places)

64

In [8]:
path = 'tickets.tsv'

with open(path, 'w', encoding='utf-8') as f:
  f.write('Điểm đi\tĐiểm đến\tSố chỗ\tGiá (VNĐ)')

  for to_place in to_places:
    # Parse trang web của từng điểm đến:
    to_place_url = to_place.a['href']
    to_place_r = requests.get(to_place_url)
    if not to_place_r.from_cache:
      time.sleep(0.5)
    to_place_tree = BeautifulSoup(to_place_r.text)

    # Lấy danh sách điểm đi ứng với điểm đến to_place:
    from_places_group = to_place_tree.find('div', {'class': 'noidungtin'})
    from_places = from_places_group.find_all('li', {'class': 'khuyenmaili'})
    #print(len(from_places))

    for from_place in from_places:
      # Parse trang web của từng điểm đi:
      from_place_url = from_place.a['href']
      from_place_r = requests.get(from_place_url)
      if not from_place_r.from_cache:
        time.sleep(0.5)
      from_place_tree = BeautifulSoup(from_place_r.text)

      # Bảng giá vé:
      from_place_table = from_place_tree.table
      rows = from_place_table.find_all('tr')[2:]

      # Điểm đi - Điểm đến
      locations = from_place.text.strip().split(' - ')

      for row in rows:
        if row.a == None: continue

        # Điểm đi:
        col2 = row.td.next_sibling.next_sibling
        departure = locations[0]
        #print(departure)

        col3 = col2.next_sibling.next_sibling
        
        # Điểm đến:
        col4 = col3.next_sibling.next_sibling
        destination = locations[1]
        #print(destination)

        # Số chỗ:
        col5 = col4.next_sibling.next_sibling
        seats = col5.string.strip()
        try:
          seats = float(re.findall('\d+', seats)[0])
        except:
          continue
        #print(seats)

        # Giá:
        tokens = row.font.text.split()
        try:
          price = float(tokens[0].replace('.', ''))
        except:
          try:
            price = float(tokens[1].replace('.', ''))
          except:
            continue
        #print(price)

        f.write(f'\n{departure}\t{destination}\t{seats}\t{price}')